In [1]:
import argparse
import json
import logging
import math
import os
import random
from pathlib import Path

import datasets
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

import evaluate
import transformers
from accelerate import Accelerator
from accelerate.logging import get_logger
from accelerate.utils import set_seed
from huggingface_hub import Repository
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    PretrainedConfig,
    SchedulerType,
    default_data_collator,
    get_scheduler,
)
from transformers.utils import check_min_version, get_full_repo_name, send_example_telemetry
from transformers.utils.versions import require_version

/home/aicenter/pyven/pytorch/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-12-22 06:31:05.310864: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-22 06:31:05.456035: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-22 06:31:06.321046: W tensorflow/compiler/xla/stream_executor/platform/default/dso_lo

In [2]:
raw_datasets = load_dataset("glue", "sst2")

Dataset glue downloaded and prepared to /home/aicenter/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 1724.40it/s]


In [6]:
raw_datasets['train'].features["label"]

ClassLabel(names=['negative', 'positive'], id=None)

In [58]:
data_files = {}
data_files["train"] = './true-false-qa.json'
cus_raw_datasets = load_dataset('json', data_files=data_files)

Using custom data configuration default-22b9ae19f86e1df3
Found cached dataset json (/home/aicenter/.cache/huggingface/datasets/json/default-22b9ae19f86e1df3/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)
100%|██████████| 1/1 [00:00<00:00, 1312.77it/s]


In [59]:
cus_raw_datasets, raw_datasets

(DatasetDict({
     train: Dataset({
         features: ['id', 'answer', 'question'],
         num_rows: 2161
     })
 }),
 DatasetDict({
     train: Dataset({
         features: ['sentence', 'label', 'idx'],
         num_rows: 67349
     })
     validation: Dataset({
         features: ['sentence', 'label', 'idx'],
         num_rows: 872
     })
     test: Dataset({
         features: ['sentence', 'label', 'idx'],
         num_rows: 1821
     })
 }))

In [22]:
cus_raw_datasets["train"].features["answer"]

Value(dtype='string', id=None)

In [75]:
raw_datasets["train"][0]

{'sentence': 'hide new secretions from the parental units ',
 'label': 0,
 'idx': 0}

In [18]:
non_label_column_names = [name for name in cus_raw_datasets["train"].column_names if name != "label"]
non_label_column_names

['id', 'answer', 'question']

In [68]:
import json 
A = json.load(open('./true-false-qa.json'))

In [69]:
import random
random.shuffle(A)

In [70]:
def modify(obj):
    obj['sentence'] = (obj['question'])
    obj['label'] = int(obj['answer'])
    obj['idx'] = int(obj['id'])
    del obj['question']
    del obj['answer']
    del obj['id']


In [71]:
for a in A:
    modify(a)

In [74]:
valid_data_json = A[:len(A)//10]
train_data_json = A[len(A)//10:]

json.dump(valid_data_json, open('./TF-random-valid.json', 'w'), indent=2, ensure_ascii=False)
json.dump(train_data_json, open('./TF-random-train.json', 'w'), indent=2, ensure_ascii=False)

In [76]:
maxL = 0

for a in A:
    if len(a['sentence']) > maxL: maxL = len(a['sentence'])

In [77]:
maxL

177